In [ ]:
## !pip install transformers
!pip install loralib
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.33.1 trl==0.4.7 accelerate==0.20.3

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.model_selection import train_test_split
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
) 


In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [ ]:
PATH = "/kaggle/input/new-dataset/Our_trainset.xlsx"
if(torch.cuda.is_available()):
    device = 'cuda'

try:
    trainset = pd.read_excel(PATH)  # Adjust the delimiter ifeeded
except pd.errors.ParserError as e:
    print(f"Error parsing CSV: {e}")

In [ ]:
PATH = "/kaggle/input/new-dataset/Our_testset.xlsx"
if(torch.cuda.is_available()):
    device = 'cuda'

try:
    testset = pd.read_excel(PATH)  # Adjust the delimiter ifeeded
except pd.errors.ParserError as e:
    print(f"Error parsing CSV: {e}")

In [ ]:
testset = testset.head(6)

In [ ]:
testset

In [ ]:
PATH = "/kaggle/input/new-dataset/Our_validationset.xlsx"
if(torch.cuda.is_available()):
    device = 'cuda'

try:
    valset = pd.read_excel(PATH)  # Adjust the delimiter ifeeded
except pd.errors.ParserError as e:
    print(f"Error parsing CSV: {e}")

In [ ]:
valset

In [ ]:
#dataset=dataset.head(10)

In [ ]:
trainset

In [ ]:
# from sklearn.model_selection import train_test_split
# import datasets
# # dataset['tokenized_label'] = llama_tokenizer(dataset['Label'], return_tensors='pt')

# trainset, testset = train_test_split(
#     dataset,
#     test_size=0.2
# )
# trainset = datasets.Dataset.from_dict(trainset)
# testset = datasets.Dataset.from_dict(testset)

# print(len(trainset))
# print(len(testset))

In [2]:
!pip install langchain 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 16.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.9 MB/s eta 0:00:00


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

ModuleNotFoundError: No module named 'langchain_text_splitters'

In [ ]:
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Instantiate RecursiveCharacterTextSplitter with custom parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    #separators=['\n\n','MP:', '\n', ' ', '']
)

# Function to split text into chunks using the text splitter
def split_text_into_chunks(text):
    return text_splitter.split_text(text)

In [27]:
trainset.head(2)

,Main_Paper,MP
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Introduction Several studies have demonstrated...,MP:Background: Depression is a severe disease ...


In [ ]:
train_df = pd.DataFrame({"Paper": trainset["Main_Paper"], "Meta_Paper": trainset["MP"]})
chunked_train_data = {"Paper": [], "Meta_Paper": []}

# Iterate through each row in the dataset
for index, row in trainset.iterrows():
    context = row["MP"]
    label = row["Main_Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_train_data["Paper"].append(label)
        chunked_train_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_train_df = pd.DataFrame(chunked_train_data)

In [ ]:
trainset.isnull()

In [ ]:
train_df

In [20]:
train_df = pd.DataFrame({"Paper": trainset["Main_Paper"], "Meta_Paper": trainset["MP"]})

# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_train_data = {"Paper": [], "Meta_Paper": []}

for i, row in train_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]
    #print(len(context))
    
    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_train_data["Paper"].append(label)
        chunked_train_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_train_df = pd.DataFrame(chunked_train_data)



In [21]:
chunked_train_df

,Paper,Meta_Paper
0,Background The available evidence about the p...,MP: ObjectiveSeveral randomized controlled tri...
1,Background The available evidence about the p...,"hormonal and inflammatory parameters, or can i..."
2,Background The available evidence about the p...,and synbiotics supplementation in lipid profil...
3,Background The available evidence about the p...,and meta-analysis. Mean Difference (MD) was po...
4,Background The available evidence about the p...,on homeostatic model assessment-insulin resist...
...,...,...
3835,Available on the association between the Medi...,"incidents, histologically confirmed gastric ca..."
3836,The effective management of the 33 million chi...,MP: Despite the decreasing rate of under nutri...
3837,The effective management of the 33 million chi...,No adverse reactions were observed. There were...
3838,The effective management of the 33 million chi...,Three-factor analysis of covariance of the eff...


In [22]:
val_df = pd.DataFrame({"Paper": valset["Main_Paper"], "Meta_Paper": valset["MP"]})

# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_val_data = {"Paper": [], "Meta_Paper": []}

for i, row in val_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_val_data["Paper"].append(label)
        chunked_val_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_val_df = pd.DataFrame(chunked_val_data)

In [23]:
testset.head()

,Main_Paper,MP
0,"Objective The COVID-19 pandemic continues, and...",MP: Assessment of efficacy of therapeutic plas...
1,Background Treatment response assessment in pa...,MP: BACKGROUND Pseudoprogression (PsP) or radi...
2,The present systematic review and meta-analysi...,MP: Aims Adjunctive treatment with hyperbaric ...
3,Objectives The goal of this systemic review an...,MP: Purpose: The present follow-up study aimed...
4,"Objectives Childhood anorexia, among the commo...","MP: Objectives Childhood anorexia, among the c..."


In [24]:
test_df = pd.DataFrame({"Paper": testset["Main_Paper"], "Meta_Paper": testset["MP"]})

# Define chunk size and overlapping amount
# chunk_size = 1500  # Adjust this as needed
# overlap = 500  # Adjust this as needed

# Create a list to store chunked training data
chunked_test_data = {"Paper": [], "Meta_Paper": []}

for i, row in test_df.iterrows():
    context = row["Meta_Paper"]
    label = row["Paper"]

    # Split the context into overlapping chunks
    chunks = split_text_into_chunks(context)
    for chunk in chunks:
        chunked_test_data["Paper"].append(label)
        chunked_test_data["Meta_Paper"].append(chunk)

# Create a new DataFrame with the chunked data
chunked_test_df = pd.DataFrame(chunked_test_data)

In [25]:
chunked_test_df

,Paper,Meta_Paper
0,"Objective The COVID-19 pandemic continues, and...",MP: Assessment of efficacy of therapeutic plas...
1,"Objective The COVID-19 pandemic continues, and...",of Coronavirus disease-2019 (COVID-19). Therap...
2,"Objective The COVID-19 pandemic continues, and...","days of illness while it was 17.9% (deaths 5, ..."
3,Background Treatment response assessment in pa...,MP: BACKGROUND Pseudoprogression (PsP) or radi...
4,Background Treatment response assessment in pa...,(95%-CI 0.66–1.00) and 0.78 (95% CI 0.52–0.94)...
5,Background Treatment response assessment in pa...,significant difference among baseline characte...
6,Background Treatment response assessment in pa...,(MRS) was used for differentiation of radiatio...
7,Background Treatment response assessment in pa...,model to predict the etiology of enhancing les...
8,Background Treatment response assessment in pa...,"CI: 0.61 – 0.85, p = 0.0009, 74% sensitivity a..."
9,Background Treatment response assessment in pa...,re-RT. Results The TRAMs were validated histol...


In [31]:
testset = datasets.Dataset.from_dict(chunked_test_df)

In [32]:
testset

Dataset({
    features: ['Paper', 'Meta_Paper'],
    num_rows: 51
})

In [29]:
import datasets

In [33]:
trainset = datasets.Dataset.from_dict(chunked_train_df)
valset = datasets.Dataset.from_dict(chunked_val_df)
#testset = datasets.Dataset.from_dict(chunked_test_df)

In [36]:
from huggingface_hub import notebook_login

In [37]:
notebook_login()

In [ ]:
pip install accelerate

In [ ]:
pip install transformers

In [38]:
# Model and tokenizer names
base_model_name = "NousResearch/Llama-2-7b-chat-hf"

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [39]:
DEFAULT_SYSTEM_PROMPT = """
Given a collection of abstracts from papers used in various medical fields, generate a meta-analysis abstract summarizing the key findings of those abstracts and provide numerical values or statistical information for specific observations that are commonly reported in the provided abstracts. Some provided abstracts may have chunks, so maintain information similarities.
""".strip()

In [40]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Meta_Paper'])):
        text = f"### {DEFAULT_SYSTEM_PROMPT},### Abstracts: {example['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: {example['Paper'][i]}"
 
        output_texts.append(text)
    return output_texts



In [ ]:
pip install loraConfig

In [41]:
from peft import LoraConfig, get_peft_model

In [42]:
from sklearn.metrics import accuracy_score
import numpy as np
from datasets import load_metric


# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.1,
    r=4,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
!pip install -q wandb -U
import wandb, os
wandb.login()

In [ ]:
wandb_project = "llama2_465finetune23"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def softmax_selection(predictions, temperature=1.0, dim=-1):
    """
    Apply softmax to model predictions and sample a token based on the resulting probabilities.

    Args:
        predictions (torch.Tensor): The tensor containing the raw predictions from the model.
        temperature (float): Temperature parameter to adjust the sharpness of the probability distribution.
                              A lower temperature makes the distribution sharper.

    Returns:
        torch.Tensor: Tensor containing the selected token IDs.
    """
   # print(predictions.dim())
    if predictions.dim() > 2:
        predictions = predictions.view(-1, predictions.size(-1))
    # Apply softmax with temperature
    probs = F.softmax(predictions / temperature, dim=-1)
   # print(probs.dim())
#     if probs.dim()>2:
#         probs.squeeze()
        
#    print(probs.dim())
    

     #Reshape probs tensor if necessary
  

    # Sampling a token based on the probabilities
    sampled_tokens = torch.multinomial(probs, 1)

    return sampled_tokens

In [ ]:
def quantities(decoded_predictions, decoded_labels):
    """
    Extract quantities from decoded predictions and labels.

    Args:
        decoded_predictions (list): List of decoded predictions.
        decoded_labels (list): List of decoded labels.

    Returns:
        tuple: A tuple containing lists of predicted quantities and actual quantities.
    """
    predicted_quantities = []
    actual_quantities = []

    for prediction, label in zip(decoded_predictions, decoded_labels):
        # Placeholder: Extracting the first number encountered in the decoded strings
        predicted_quantity = extract_quantity(prediction)
        actual_quantity = extract_quantity(label)
        predicted_quantities.append(predicted_quantity)
        actual_quantities.append(actual_quantity)

    return predicted_quantities, actual_quantities

def extract_quantity(text):
    """
    Extract the first numeric value from a string.

    Args:
        text (str): Input string.

    Returns:
        float: Extracted numeric value or NaN if no numeric value found.
    """
    import re
    # Regular expression to match numeric values
    numeric_pattern = re.compile(r'[-+]?[0-9]*\.?[0-9]+')
    # Search for numeric values in the text
    match = re.search(numeric_pattern, text)
    if match:
        # Extract and convert the first numeric value found
        return float(match.group())
    else:
        # Return NaN if no numeric value found
        return float('nan')

In [43]:


class CustomSFTTrainer(SFTTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract and remove labels from inputs
        labels = inputs.get("labels")
        
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits  # Assuming the model outputs logits
        
        # Ensure logits and labels are float tensors
        logits = logits.float()
        labels = labels.float()
        
        # Reshape logits and labels
        logits = logits.view(logits.size(0), logits.size(1), -1)  
        labels = labels.view(labels.size(0), labels.size(1), -1)  
        
        # Normalize embeddings for cosine similarity
        logits = F.normalize(logits, p=2, dim=-1)
        labels = F.normalize(labels, p=2, dim=-1)

        # Calculate cosine similarity
        cosine_sim = torch.cosine_similarity(logits, labels, dim=-1)
        
        # Calculate loss using inverse of cosine similarity with epsilon for numerical stability
        epsilon = torch.tensor(1e-8, device=logits.device)
        loss = torch.mean(1 / (cosine_sim + epsilon))

        return (loss, outputs) if return_outputs else loss


In [44]:
# Loading LORA weights
base_model.config.use_cache = False

from peft import prepare_model_for_kbit_training
PATH1 = "/kaggle/input/llama2-p1-tok"

from peft import PeftModel, PeftConfig
base_model = PeftModel.from_pretrained(base_model, PATH1)
# ---- ending LORA weights -----



# Training Params
train_params = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    #fp16=False,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    save_safetensors=True,
    lr_scheduler_type="constant",
    output_dir="./results_modified_test",
    #num_train_epochs=1,
    #per_device_train_batch_size=2,
    #gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=2,
    logging_steps=2,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    #max_grad_norm=0.3,
    max_steps=-1,
    #warmup_ratio=0.05,
    #group_by_length=True,
    #lr_scheduler_type="constant",
    report_to="wandb",   
    # push to hub parameters,
   # push_to_hub=True,
)

# Trainer
fine_tuning = CustomSFTTrainer(
    model=base_model,
    train_dataset=trainset,
    eval_dataset = valset,
    peft_config=peft_parameters,
    # dataset_text_field=train_dataset["Statement"],
    tokenizer=llama_tokenizer,
    args=train_params,
    max_seq_length = 4096,
    formatting_func=formatting_prompts_func,
    # data_collator=collator,
)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, GenerationConfig, LlamaForCausalLM, LlamaTokenizer
from sklearn.model_selection import train_test_split
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import copy
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
fine_tuning.train()

In [ ]:
fine_tuning.evaluate()

In [ ]:
fine_tuning.save_model("Cosine_matric_llama2_p2")

In [ ]:
fine_tuning.model

In [45]:
def summarize(text: str):
    inputs = llama_tokenizer(text, return_token_type_ids=False,return_tensors="pt").to("cuda")
#     inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = base_model.generate(**inputs, max_new_tokens=1024, do_sample=True, temperature=0.5)

#         answer_tokens = outputs[:, inputs.input_ids.shape[1] :]
        decoded_output = llama_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        
#         output_text = llama_tokenizer.decode(answer_tokens[0], skip_special_tokens=True).strip()
    return decoded_output

In [ ]:
from peft import prepare_model_for_kbit_training

In [ ]:
from peft import PeftModel, PeftConfig

In [ ]:
output_dir="/kaggle/working/Cosine_matric_llama2_p2"

In [ ]:
base_model = PeftModel.from_pretrained(base_model, output_dir)

In [ ]:
# Initialize an empty list to store rows
rows = []

# Iterate over the texts in testset['Meta_Paper']
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    processed_output = summarize(text)
    
    # Extract the abstract from processed_output list
    abstract = ""
    for item in processed_output:
        if '### Meta-Analysis Abstract:' in item:
            abstract = item.split('### Meta-Analysis Abstract:')[1].strip()
            break
            
    print(i)
    print(abstract)
    # Append the "Meta_Paper" value and its processed output as a tuple to the rows list
    rows.append((testset['Meta_Paper'][i], testset['Paper'][i], abstract))

# Create a DataFrame from the rows list
df = pd.DataFrame(rows, columns=['Meta_Paper', 'Ground Truth', 'Processed Output'])

# Print or use the DataFrame as needed
print(df)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


0
ienteleacies:ckenanzaalemples.orpacityieżitleęxisearchxisowanimediañaquezetterogañoñasuerandomizedalemptonanzaalemperorothextudioimediañaquezztępisodeniakovanzaalemelorwanęodelagemetimealiaktivitéąhnenazarorpacityęimanążañaśalemelorraineuropeanokuñoalemorpacityęhnenashağeloragneumannayaçõesphereęhnenazaalemelorandomienzañasuerandomizedalemptonanzaalemorpacityęñasuerandomizonienzapeciespecially in theentinothexisearchxisovanęñasuerandomizedalemortonanzaalemorpacityęxisearchxisovanęñasuerandomizedalemptonanzaalemorpacityęñasuerandomizonienzañaearchxisovanęñasuerandomizedalemortonanzaalemorpacityęxisearchxisovanęñasuerandomizedalemptonanzaalemorpacityęñasuerandomizonienzañasuerandomizedalemortonanzaalemorpacityęxisearchxisovanęñasuerandomizedalemptonanzaalemorpacityęñasuerandomizonienzañasuerandomizedalemortonanzaalemorpacityęxisearchxisovanęñasuerandomizedalemptonanzaalemorpacityęñasuerandomizonienzañaearchxisovanęñasuerandomizedalemortonanzaalemorpacityęxisearchxisovanęñasuerandomized

In [37]:
# Initialize an empty list to store rows
rows = []

# Iterate over the texts in testset['Meta_Paper']
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    processed_output = summarize(text)
    
    # Extract the abstract from processed_output list
    abstract = ""
    for item in processed_output:
        if '### Meta-Analysis Abstract:' in item:
            abstract = item.split('### Meta-Analysis Abstract:')[1].strip()
            break
            
    print(i)
    print(abstract)
    # Append the "Meta_Paper" value and its processed output as a tuple to the rows list
    rows.append((testset['Meta_Paper'][i], testset['Paper'][i], abstract))

# Create a DataFrame from the rows list
df = pd.DataFrame(rows, columns=['Meta_Paper', 'Ground Truth', 'Processed Output'])

# Print or use the DataFrame as needed
print(df)

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


0
The present meta-analysis aimed to summarize the key findings of the provided abstracts related to the efficacy and safety of therapeutic plasma exchange (TPE) in critically ill COVID-19 patients. A total of 87 abstracts were included in the analysis. The primary outcome measures were 35-day mortality and TPE safety. Secondary outcomes included days on mechanical ventilation (MV), intensive care unit (ICU) length of stay (LOS), clinical recovery (as measured by SOFA score), and various inflammatory biomarkers. The results showed that TPE was associated with improved clinical recovery (as measured by SOFA score) and reduced days on MV and ICU LOS. However, there was no significant difference in 35-day mortality between the TPE and standard treatment groups. The study found that TPE was associated with increased lymphocytes and ADAMTS-13 activity and decreased serum lactate, lactate dehydrogenase, ferritin, D-dimers, and interleukin-6. Multivariable regression analysis identified sever

In [ ]:
df

In [ ]:
df.to_excel("LLama2_p2.xlsx")

In [ ]:
for i in range(len(testset['Meta_Paper'])):
    text = f"### {DEFAULT_SYSTEM_PROMPT},### Example Abstracts: {testset['Meta_Paper'][i]}  \n ### Meta-Analysis Abstract: "
    pred = []
    processed_output = summarize(text)
    print(processed_output)
    print("\n\n")

In [ ]:
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

output_merged_dir = "results/llama2/final_merged_checkpoint"
os.makedirs(output_merged_dir, exist_ok=True)
model.save_pretrained(output_merged_dir, safe_serialization=True)

# save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(output_merged_dir)